### Checkpoints

S1=420.4 g/day

H:C in food  1.8528 (mol basis)

N:C in food  0.0957 (mol basis)

Q3= Q4 = 5.514 l/day - make sure you get this when calculating unit 3 feed

Numbers below will depend on on your digester volumes

CNH3 (in Q3 before entering unit 4) - 145.3 mmol/L (for open system)

CP (in Q3 before entering unit 4) - 10.8 mmol/L (for open system)

CNa (in Q3 before entering unit 4) - 21.8 mmol/L (for open system)

CNH3 (in Q4 before entering unit 5) - 153 mmol/L (for open system)

CP (in Q4 before entering unit 5) - 13 mmol/L (for open system)

CNa (in Q4 before entering unit 5) - 21.3 mmol/L (for open system)

In [686]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

### Section 1: Calculating food masses

In [687]:
kilojoulesRequiredPerDay = 9000
#       fat     carbs       protein
foodMassFraction = np.array([0.21, 0.43, 0.36])
energyPerFoodGroup = np.array([38, 17, 17])
energyPerDryGramFood = np.sum(foodMassFraction*energyPerFoodGroup)
massS1 = kilojoulesRequiredPerDay / energyPerDryGramFood

In [688]:
print('Dry grams of edible food per day =', np.round(massS1,2))
print('Expected: 420.4 grams per day')

Dry grams of edible food per day = 420.36
Expected: 420.4 grams per day


In [689]:
fatFormula = np.array([1,2,0.11,0])
carbsFormula = np.array([1,2,1,0])
proteinFormula = np.array([1,1.6,0.32,0.26])
MMChon = np.array([12,1,16,14])
MM = np.array([12,1,16,14,31,23])

In [690]:
molFat = foodMassFraction[0]/np.sum(fatFormula*MMChon)
molCarbs = foodMassFraction[1]/np.sum(carbsFormula*MMChon)
molProtein = foodMassFraction[2]/np.sum(proteinFormula*MMChon)

totalMolsPerGramFood = molProtein + molFat + molCarbs

proteinMolFrac = molProtein / totalMolsPerGramFood
fatMolFrac = molFat / totalMolsPerGramFood
carbsMolFrac = molCarbs / totalMolsPerGramFood

CinFood = proteinMolFrac*proteinFormula[0] + fatMolFrac*fatFormula[0] + carbsMolFrac*carbsFormula[0]
HinFood = proteinMolFrac*proteinFormula[1] + fatMolFrac*fatFormula[1] + carbsMolFrac*carbsFormula[1]
OinFood = proteinMolFrac*proteinFormula[2] + fatMolFrac*fatFormula[2] + carbsMolFrac*carbsFormula[2]
NinFood = proteinMolFrac*proteinFormula[3] + fatMolFrac*fatFormula[3] + carbsMolFrac*carbsFormula[3]
print(CinFood, HinFood, OinFood, NinFood)

1.0000000000000002 1.8528265723435982 0.47879112690048864 0.09566272797666145


In [691]:
S1 = np.array([CinFood, HinFood, OinFood, NinFood, 0.0036, 0.0016])
S2 = np.array([1, 1.75, 0.59, 0.125, 0.005825, 0.00172])
S3 = np.array([1, 1.46, 0.6, 0.02, 0.0025, 0.0023])
X3 = np.array([1,1.8,0.5,0.08,0.02,0])
S4 = X4 = np.array([1,1.7,0.55,0.13,0.02,0])
S5 = np.array([1, 1.7, 0.75, 0.02, 0.00275, 0.00118])
Urea = np.array([1,4,1,2])


In [692]:
massS3 = massS1 / 0.4 * 0.6
waterInS1 = massS1 / 0.4 * 0.6 / 1000
waterInS3 = massS3 / 0.4 * 0.6 / 1000

molsS1 = massS1 / np.sum(S1*MM)
molsS3 = massS3 / np.sum(S3*MM)

In [693]:
print(waterInS1)

0.6305464736104622


### Section 2: Hydroponics

In [694]:
                        #  S1   ,S3   ,CO2  ,H2O  ,O2   ,NH3  ,P    ,Na   
unit1MatrixA = np.matrix([[S1[0],S3[0],1    ,0    ,0    ,0    ,0    ,0    ],    #C
                          [S1[1],S3[1],0    ,2    ,0    ,3    ,0    ,0    ],    #H
                          [S1[2],S3[2],2    ,1    ,2    ,0    ,0    ,0    ],    #O
                          [S1[3],S3[3],0    ,0    ,0    ,1    ,0    ,0    ],    #N
                          [S1[4],S3[4],0    ,0    ,0    ,0    ,1    ,0    ],    #P
                          [S1[5],S3[5],0    ,0    ,0    ,0    ,0    ,1    ],    #Na
                          [1    ,0    ,0    ,0    ,0    ,0    ,0    ,0    ],    #S1 rate
                          [0    ,1    ,0    ,0    ,0    ,0    ,0    ,0    ]     #S3 rate
                          ])

unit1MatrixR = np.matrix([[0, 0, 0, 0, 0, 0, molsS1, molsS3]]).T
unit1Rates = np.linalg.solve(unit1MatrixA, unit1MatrixR)
rs1U1, rs3U1, rco2U1, rh2oU1, ro2U1, rnh3U1, rpU1, rnaU1 = unit1Rates.flat

### Section 3: Human

In [695]:
molsUrea = 40 / np.sum(Urea*MMChon)

In [696]:
                        #  S1   ,S2   ,CO2  ,H2O  ,O2   ,P    ,Na   ,U
unit2MatrixA = np.matrix([[S1[0],S2[0],1    ,0    ,0    ,0    ,0    ,1],    #C
                          [S1[1],S2[1],0    ,2    ,0    ,0    ,0    ,4],    #H
                          [S1[2],S2[2],2    ,1    ,2    ,0    ,0    ,1],    #O
                          [S1[3],S2[3],0    ,0    ,0    ,0    ,0    ,2],    #N
                          [S1[4],S2[4],0    ,0    ,0    ,1    ,0    ,0],    #P
                          [S1[5],S2[5],0    ,0    ,0    ,0    ,1    ,0],    #Na
                          [-1   ,0    ,0    ,0    ,0    ,0    ,0    ,0],    #S1 rate
                          [0    ,0    ,0    ,0    ,0    ,0    ,0    ,1]     #U rate
                          ])

unit2MatrixR = np.matrix([[0, 0, 0, 0, 0, 0, molsS1, molsUrea]]).T
unit2Rates = np.linalg.solve(unit2MatrixA, unit2MatrixR)
rs1U2, rs2U2, rco2U2, rh2oU2, ro2U2, rpU2, rnaU2, ruU2 = unit2Rates.flat

### Section 4: Preparing for unit 3 & 4

Outline to follow:
* Calculate substrate formula for unit 3
* Calculate water requirements for unit 3


In [697]:
thetamax3=0.55 # Unit 3
mumax3=0.07 #1/day
Ks_mu3=0.04 #cmol/L
Ks_theta3=0.008/1e6  #cmol/L
Na_conc=300/1000/23 #mol/L (300mg/L in clean water)
mumax4=0.17 #1/day
thetamax4=0.57 #molATP/cmolX/day in Unit4
kla=20 #1/day
Ks_mu4=0.002 #cmol/L
Co_sat=7/1000/32 #mol/l
Ko_mu4=Co_sat*0.15 #mol/L
Ks_theta4=Ks_mu4/1e6 #cmol/L
Ko_theta4=Ko_mu4/1e6 #mol/L
Q1=1.5  #L

In [698]:
err = 1
D3 = 1.26/3 * mumax3
D4 = 0.739/3 * mumax4
guessX4 = 0

#------------------------------------------------Calculation of Q3---------------------------------------------------------------------
while err > 1e-12:    
    molsSU3 = molsS3 + rs2U2 + guessX4
    SU3TotalMolarComposition = molsS3*S3 + rs2U2*S2 + guessX4*X4
    SU3 = SU3TotalMolarComposition / SU3TotalMolarComposition[0]
    massSU3 = molsSU3 * np.sum(MM*SU3)
    requiredInletConcentration = 130 #g/L
    Q3 = Q4 = massSU3 / requiredInletConcentration
    existingWater = (waterInS3 + rs2U2*np.sum(MM*S2)/0.25*0.75/1000)
    waterToBeAddedBeforeRecycle = Q3 - existingWater
    print('The calculated value for Q3 =',round(Q3,3))
    print('Expected: 5.514 L')
    print('\nThe water which needs to be added to unit 3 before the recycle =',waterToBeAddedBeforeRecycle)
    #--------------------------------------------------Solving Unit 3---------------------------------------------------------------------
                            #  S     ,X    ,R    ,CH4  ,CO2  ,H2O  ,NH3  ,P    ,Na   
    unit3MatrixA = np.matrix([[SU3[0],X3[0],S5[0],1    ,1    ,0    ,0    ,0    ,0    ],    #C
                              [SU3[1],X3[1],S5[1],4    ,0    ,2    ,3    ,0    ,0    ],    #H
                              [SU3[2],X3[2],S5[2],0    ,2    ,1    ,0    ,0    ,0    ],    #O
                              [SU3[3],X3[3],S5[3],0    ,0    ,0    ,1    ,0    ,0    ],    #N
                              [SU3[4],X3[4],S5[4],0    ,0    ,0    ,0    ,1    ,0    ],    #P
                              [SU3[5],X3[5],S5[5],0    ,0    ,0    ,0    ,0    ,1    ],    #Na
                              [-0.35 ,-1.8 ,0    ,0    ,0    ,0    ,0    ,0    ,0    ],    #theta
                              [0     ,1    ,0    ,0    ,0    ,0    ,0    ,0    ,0    ],    #mu
                              [0.18  ,0    ,1    ,0    ,0    ,0    ,0    ,0    ,0    ]     #S to R ratio
                              ])
    #--------------------------------------------------Steady state equations unit 3---------------------------------------------------------------------

    def responseFunctionUnit3(Cs):
        mu3 = mumax3 * Cs/(Ks_mu3 + Cs)
        theta3 = thetamax3 * Cs/(Ks_theta3 + Cs)
        unit3MatrixR = np.matrix([[0,0,0,0,0,0,theta3,mu3,0]]).T
        r1Unit3 = np.linalg.solve(unit3MatrixA,unit3MatrixR)
        return r1Unit3              # Order is S:[0]    X:[1]   R:[2]   CH4:[3]   Co2:[4]    H2O:[5]   NH3:[6]    P:[7]    Na:[8]

    Csf3, Cxf3, Crf3, Cnh3f3, Cpf3, Cnaf3 = [requiredInletConcentration/np.sum(MM*SU3), 0, 0, 0, 0, Na_conc*waterToBeAddedBeforeRecycle/Q3]

    def steadyStateUnit3(C):
        Cs, Cx, Cr, Cnh3, Cp, Cna = C
        rs, rx, rr, rch4, rco2, rh2o, rnh3, rp, rna = responseFunctionUnit3(Cs).flat
        return[
            D3*(Csf3 - Cs) + rs*Cx,
            D3*(Cxf3 - Cx) + rx*Cx,
            D3*(Crf3 - Cr) + rr*Cx,
            D3*(Cnh3f3 - Cnh3) + rnh3*Cx,
            D3*(Cpf3 - Cp) + rp*Cx,
            D3*(Cnaf3 - Cna) + rna*Cx
        ]
    #---------------------------------------------------------------------------------------------------------------------------------------------
    guess3 = [0.01, 0.01, 0.95, 0.146, 0.011, 0.021]
    steadyState3 = fsolve(steadyStateUnit3,guess3)
    Cs3Final, Cx3Final, Cr3Final, Cnh33Final, Cp3Final, Cna3Final = steadyState3.flat

    #--------------------------------------------------Substrate going into 4---------------------------------------------------------------------
    molsSU4 = Cs3Final + Cx3Final
    SU4TotalMolarComposition = Cs3Final*SU3 + Cx3Final*X3
    SU4 = SU4TotalMolarComposition / SU4TotalMolarComposition[0]
    #--------------------------------------------------Steady state equations unit 4----------------------------------------------------------------

                            #  S     ,O2   ,X    ,CO2  ,H2O  ,NH3  ,P    ,Na   
    unit4MatrixA = np.matrix([[SU4[0],0    ,X4[0],1    ,0    ,0    ,0    ,0    ],    #C
                              [SU4[1],0    ,X4[1],0    ,2    ,3    ,0    ,0    ],    #H
                              [SU4[2],2    ,X4[2],2    ,1    ,0    ,0    ,0    ],    #O
                              [SU4[3],0    ,X4[3],0    ,0    ,1    ,0    ,0    ],    #N
                              [SU4[4],0    ,X4[4],0    ,0    ,0    ,1    ,0    ],    #P
                              [SU4[5],0    ,X4[5],0    ,0    ,0    ,0    ,1    ],    #Na
                              [0     ,-3   ,-2.5 ,0    ,0    ,0    ,0    ,0    ],    #theta
                              [0     ,0    ,1    ,0    ,0    ,0    ,0    ,0    ]     #mu
                              ])

    def responseFunctionUnit4(Cs,Co):
        mu4 = mumax4*Cs/(Ks_mu4+Cs)*(Co/(Ko_mu4+Co))
        theta4 = thetamax4*Cs/(Ks_theta4+Cs)*(Co/(Ko_theta4+Co))
        unit4MatrixR = np.matrix([[0,0,0,0,0,0,theta4,mu4]]).T
        r1Unit4 = np.linalg.solve(unit4MatrixA,unit4MatrixR)
        return r1Unit4

    Csf4, Cof4, Cxf4, Cnh3f4, Cpf4, Cnaf4 = [molsSU4, Co_sat/2, 0, Cnh33Final, Cp3Final, Cna3Final]

    def steadyStateUnit4(C):
        Cs, Co, Cx, Cnh3, Cp, Cna = C
        rs, ro, rx, rco2, rh2o, rnh3, rp, rna = responseFunctionUnit4(Cs,Co).flat   
        return [
        rs*Cx + D4*(Csf4-Cs),
        ro*Cx + kla*(Co_sat-Co),
        rx*Cx + D4*(Cxf4-Cx),
        rnh3*Cx + D4*(Cnh3f4-Cnh3),
        rp*Cx + D4*(Cpf4-Cp),
        rna*Cx + D4*(Cnaf4-Cna)
        ]
    #----------------------------------------------------------------------------------------------------------------------------------------------------
    guess4 = [0.09, 2.42e-5, 0.018, 0.15, 0.02, 0.01]
    steadyState4 = fsolve(steadyStateUnit4,guess4)
    Cs4Final, Co4Final, Cx4Final, Cnh34Final, Cp4Final, Cna4Final = steadyState4.flat

    #-----------------------------------------------------Closing while loop--------------------------------------------------------------------
    err = guessX4 - Cx4Final
    guessX4 = Cx4Final

The calculated value for Q3 = 5.493
Expected: 5.514 L

The water which needs to be added to unit 3 before the recycle = 4.296473343488871


In [699]:
print('Total water required =',round(Q3,4))
print('Expected: 5.514')
print('\nCNh3 in 3 = {}; CP in 3 = {}; CNa in 3 = {}'.format(np.round(Cnh33Final*1000,1),np.round(Cp3Final*1000,1),np.round(Cna3Final*1000,1)))
print('Expected: CNh3 in 3 = 145.3; CP in 3 = 10.8; CNa in 3 = 21.8')
print('\nCNh3 in 4 = {}; CP in 4 = {}; CNa in 4 = {}'.format(np.round(Cnh34Final*1000+Cs4Final*SU4[3]*1000,1),np.round(Cp4Final*1000+Cs4Final*SU4[4]*1000,1),np.round(Cna4Final*1000+Cs4Final*SU4[5]*1000,1)))
print('Expected: CNh3 in 4 = 153; CP in 4 = 13; CNa in 4 = 21.3')
print('\nMass of substrate leaving unit 4 =',Cs4Final*np.sum(MM*SU4))
print('\nBiomass formed in unit 4 =',np.round(Cx4Final,4))


Total water required = 5.4929
Expected: 5.514

CNh3 in 3 = 145.3; CP in 3 = 11.1; CNa in 3 = 21.3
Expected: CNh3 in 3 = 145.3; CP in 3 = 10.8; CNa in 3 = 21.8

CNh3 in 4 = 151.3; CP in 4 = 12.7; CNa in 4 = 21.3
Expected: CNh3 in 4 = 153; CP in 4 = 13; CNa in 4 = 21.3

Mass of substrate leaving unit 4 = 0.39907601435320944

Biomass formed in unit 4 = 0.0183


In [700]:
Cna4Final*1000

21.321271733719048

In [701]:
V3 = Q3/D3
V4 = Q4/D4
Vtot = V3 + V4
print(Vtot)

318.00038561090383


In [702]:
rnaU1*1000

-91.03214602478292

In [703]:
#Write down the minimum volume below:
#318.00038561090383

In [704]:
unit3PBalance = Csf3*Q3*SU3[4] - Cs3Final*Q3*SU3[4] - Cx3Final*Q3*X3[4] - Cr3Final*Q3*S5[4]  - Cp3Final*Q3
unit3NaBalance = Csf3*Q3*SU3[5] + Cnaf3*Q3 - Cs3Final*Q3*SU3[5] - Cx3Final*Q3*X3[5] - Cr3Final*Q3*S5[5]  - Cna3Final*Q3
unit3NBalance = Csf3*Q3*SU3[3] - Cs3Final*Q3*SU3[3] - Cx3Final*Q3*X3[3] - Cr3Final*Q3*S5[3] - Cnh33Final*Q3
print(unit3PBalance)
print(unit3NaBalance)
print(unit3NBalance)

2.0816681711721685e-17
1.3877787807814457e-17
-1.1102230246251565e-16


In [705]:
unit4PBalance = Cpf4*Q4 + Csf4*Q4*SU4[4] - Cp4Final*(Q4) - Cx4Final * X4[4] * Q4 - Cs4Final*Q4*SU4[4]
unit4NaBalance = Cnaf4*Q4 + Csf4*Q4*SU4[5] - Cna4Final*Q4 - Cs4Final*Q4*SU4[5]
unit4NBalance = Csf4*Q4*SU4[3] + Cnh3f4*Q4 - Cs4Final*Q4*SU4[3] - Cx4Final*Q4*X4[3] - Cnh34Final*Q4
print(unit4PBalance)
print(unit4NaBalance)
print(unit4NBalance)

4.9873299934333204e-18
-3.9776667203061944e-18
-1.1102230246251565e-16


In [706]:
err1 = 1
err2 = 1
err3 = 1
#Lp = 0.7
L5 = 4

#------------------------------------------------Calculation of Q3---------------------------------------------------------------------


waterInPlants = (massS1 / 0.4*0.6 + massS3/0.4*0.6)/1000
L1 = Q1
L4 = Q4
Lagri = waterInPlants - rh2oU1*18/1000 + (massS1+massS3)/0.4*0.03/1000
#L5 = L1 + L4 - Lp - Lagri
Lp = L1 + L4 - Lagri - L5
L2 = Q3 - L5 - existingWater

PEnteringS0 = 0.014965
NaEnteringS0 = 0.00178
NEnteringS0 = 0.108359

PLeavingS5 = Cr3Final*Q3*S5[4]
NaLeavingS5 = Cr3Final*Q3*S5[5]
NLeavingS5 = Cr3Final*Q3*S5[3]

CpU5 = (PEnteringS0 - PLeavingS5) / Lp
CnaU5 = (NaEnteringS0 - NaLeavingS5 + Na_conc*L2) / Lp
CNh3U5 = (NEnteringS0 - NLeavingS5) / Lp

#--------------------------------------------------Solving Unit 3---------------------------------------------------------------------
                        #  S     ,X    ,R    ,CH4  ,CO2  ,H2O  ,NH3  ,P    ,Na   
unit3MatrixA = np.matrix([[SU3[0],X3[0],S5[0],1    ,1    ,0    ,0    ,0    ,0    ],    #C
                          [SU3[1],X3[1],S5[1],4    ,0    ,2    ,3    ,0    ,0    ],    #H
                          [SU3[2],X3[2],S5[2],0    ,2    ,1    ,0    ,0    ,0    ],    #O
                          [SU3[3],X3[3],S5[3],0    ,0    ,0    ,1    ,0    ,0    ],    #N
                          [SU3[4],X3[4],S5[4],0    ,0    ,0    ,0    ,1    ,0    ],    #P
                          [SU3[5],X3[5],S5[5],0    ,0    ,0    ,0    ,0    ,1    ],    #Na
                          [-0.35 ,-1.8 ,0    ,0    ,0    ,0    ,0    ,0    ,0    ],    #theta
                          [0     ,1    ,0    ,0    ,0    ,0    ,0    ,0    ,0    ],    #mu
                          [0.18  ,0    ,1    ,0    ,0    ,0    ,0    ,0    ,0    ]     #S to R ratio
                          ])
#--------------------------------------------------Steady state equations unit 3---------------------------------------------------------------------
def responseFunctionUnit3(Cs):
    mu3 = mumax3 * Cs/(Ks_mu3 + Cs)
    theta3 = thetamax3 * Cs/(Ks_theta3 + Cs)
    unit3MatrixR = np.matrix([[0,0,0,0,0,0,theta3,mu3,0]]).T
    r1Unit3 = np.linalg.solve(unit3MatrixA,unit3MatrixR)
    return r1Unit3              # Order is S:[0]    X:[1]   R:[2]   CH4:[3]   Co2:[4]    H2O:[5]   NH3:[6]    P:[7]    Na:[8]
Csf3, Cxf3, Crf3, Cnh3f3, Cpf3, Cnaf3 = [requiredInletConcentration/np.sum(MM*SU3), 0, 0, 0+CNh3U5*L5, 0+CpU5*L5, Na_conc*L2+CnaU5*L5]
def steadyStateUnit3(C):
    Cs, Cx, Cr, Cnh3, Cp, Cna = C
    rs, rx, rr, rch4, rco2, rh2o, rnh3, rp, rna = responseFunctionUnit3(Cs).flat
    return[
        D3*(Csf3 - Cs) + rs*Cx,
        D3*(Cxf3 - Cx) + rx*Cx,
        D3*(Crf3 - Cr) + rr*Cx,
        D3*(Cnh3f3 - Cnh3) + rnh3*Cx,
        D3*(Cpf3 - Cp) + rp*Cx,
        D3*(Cnaf3 - Cna) + rna*Cx
    ]
#---------------------------------------------------------------------------------------------------------------------------------------------
guess3 = [0.01, 0.01, 0.95, 0.146, 0.011, 0.021]
steadyState3 = fsolve(steadyStateUnit3,guess3)
Cs3Final, Cx3Final, Cr3Final, Cnh33Final, Cp3Final, Cna3Final = steadyState3.flat
#--------------------------------------------------Substrate going into 4---------------------------------------------------------------------
molsSU4 = Cs3Final + Cx3Final
SU4TotalMolarComposition = Cs3Final*SU3 + Cx3Final*X3
SU4 = SU4TotalMolarComposition / SU4TotalMolarComposition[0]
#--------------------------------------------------Steady state equations unit 4----------------------------------------------------------------
                        #  S     ,O2   ,X    ,CO2  ,H2O  ,NH3  ,P    ,Na   
unit4MatrixA = np.matrix([[SU4[0],0    ,X4[0],1    ,0    ,0    ,0    ,0    ],    #C
                          [SU4[1],0    ,X4[1],0    ,2    ,3    ,0    ,0    ],    #H
                          [SU4[2],2    ,X4[2],2    ,1    ,0    ,0    ,0    ],    #O
                          [SU4[3],0    ,X4[3],0    ,0    ,1    ,0    ,0    ],    #N
                          [SU4[4],0    ,X4[4],0    ,0    ,0    ,1    ,0    ],    #P
                          [SU4[5],0    ,X4[5],0    ,0    ,0    ,0    ,1    ],    #Na
                          [0     ,-3   ,-2.5 ,0    ,0    ,0    ,0    ,0    ],    #theta
                          [0     ,0    ,1    ,0    ,0    ,0    ,0    ,0    ]     #mu
                          ])
def responseFunctionUnit4(Cs,Co):
    mu4 = mumax4*Cs/(Ks_mu4+Cs)*(Co/(Ko_mu4+Co))
    theta4 = thetamax4*Cs/(Ks_theta4+Cs)*(Co/(Ko_theta4+Co))
    unit4MatrixR = np.matrix([[0,0,0,0,0,0,theta4,mu4]]).T
    r1Unit4 = np.linalg.solve(unit4MatrixA,unit4MatrixR)
    return r1Unit4
Csf4, Cof4, Cxf4, Cnh3f4, Cpf4, Cnaf4 = [molsSU4, Co_sat/2, 0, Cnh33Final, Cp3Final, Cna3Final]
def steadyStateUnit4(C):
    Cs, Co, Cx, Cnh3, Cp, Cna = C
    rs, ro, rx, rco2, rh2o, rnh3, rp, rna = responseFunctionUnit4(Cs,Co).flat   
    return [
    rs*Cx + D4*(Csf4-Cs),
    ro*Cx + kla*(Co_sat-Co),
    rx*Cx + D4*(Cxf4-Cx),
    rnh3*Cx + D4*(Cnh3f4-Cnh3),
    rp*Cx + D4*(Cpf4-Cp),
    rna*Cx + D4*(Cnaf4-Cna)
    ]
#----------------------------------------------------------------------------------------------------------------------------------------------------
guess4 = [0.09, 2.42e-5, 0.018, 0.15, 0.02, 0.01]
steadyState4 = fsolve(steadyStateUnit4,guess4)
Cs4Final, Co4Final, Cx4Final, Cnh34Final, Cp4Final, Cna4Final = steadyState4.flat
#-----------------------------------------------------Closing while loop--------------------------------------------------------------------
# err1 = unit5PBalance
# err2 = unit5NaBalance
# err3 = unit5NBalance
print(Lp)
#Overall concentrations:
molNL1 = 40/np.sum(MMChon*Urea)*2   #mol N / day
molNL4 = Cnh34Final*L4 + Cs4Final*L4*S4[3]   #mol N / day
molPL4 = Cp4Final*Q4 + Cs4Final*L4*S4[4]     #mol P / day
#Minerals entering and leaving:


# PEntering = PLeavingS5 + CpU5*Lp
# NaEntering = NaLeaving + CnaU5*Lp - Na_conc*L2
# NEntering = NLeaving + CNh3U5*Lp

unit5PBalance = molPL4 + PEnteringS0 + rpU1 + rpU2 - CpU5*Lp - CpU5*L5
unit5NaBalance = Cna4Final*Q4 + NaEnteringS0 + rnaU1 + rnaU2 - CnaU5*Lp - CnaU5*L5
unit5NBalance = NEnteringS0 + molNL1 + molNL4 + rnh3U1 - CNh3U5*Lp - CNh3U5*L5
print(unit5PBalance)
print(unit5NaBalance)
print(unit5NBalance)
print(L2)

0.7416313663279155
0.0008332814561855958
-0.0003780549542769735
-0.0009778767271872687
0.29647334348887067


In [707]:
CnaU5*L5

-0.003940221126332691

In [708]:
print(PLeavingS5 - PEnteringS0)
print(NLeavingS5 - NEnteringS0 + Lp*CNh3U5  )

-0.00010196894994744972
0.0


In [738]:
SU4

array([1.00000000e+00, 1.72700475e+00, 5.23431077e-01, 6.85209290e-02,
       1.59406002e-02, 5.29795061e-04])

465